read data

In [1]:
import json

with open('F:/QA_with_NN/here_is_your_answer/dev-v2.0.json') as f:
    data = json.load(f)

get data in dataframe

In [2]:
train = data

train_data_list = []
context = []
question = []
answer = []
answer_start = []
is_impossible = []
example = []

for i in range(len(train['data'])):
    for j in range(len(train['data'][i]['paragraphs'])):
        context.append(train['data'][i]['paragraphs'][j]['context'])
        for k in range(len(train['data'][i]['paragraphs'][j]['qas'])):
            question.append(train['data'][i]['paragraphs'][j]['qas'][k]['question'])
            is_impossible.append(train['data'][i]['paragraphs'][j]['qas'][k]['is_impossible'])
            for l in range(len(train['data'][i]['paragraphs'][j]['qas'][k]['answers'])):
                answer.append(train['data'][i]['paragraphs'][j]['qas'][k]['answers'][l]['text'])
                answer_start.append(train['data'][i]['paragraphs'][j]['qas'][k]['answers'][l]['answer_start'])
               

for i in range(len(train['data'])):
    for j in range(len(train['data'][i]['paragraphs'])):
        context_1 = train['data'][i]['paragraphs'][j]['context']
        for k in range(len(train['data'][i]['paragraphs'][j]['qas'])):
            question_1 = train['data'][i]['paragraphs'][j]['qas'][k]['question']
            is_impossible_1 = train['data'][i]['paragraphs'][j]['qas'][k]['is_impossible']
            if is_impossible_1 == False:
                for l in range(len(train['data'][i]['paragraphs'][j]['qas'][k]['answers'])):
                    answer_1 = train['data'][i]['paragraphs'][j]['qas'][k]['answers'][l]['text']
                    answer_start_1 = train['data'][i]['paragraphs'][j]['qas'][k]['answers'][l]['answer_start']
            else:  
                answer_1 = ""
                answer_start_1= ""
            example.append((context_1, question_1, is_impossible_1, answer_1, answer_start_1))


check data

In [3]:
example[1]
#  context, question, is_impossible, answer, answer_start


('The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'When were the Normans in Normandy?',
 False,
 '10th and 11th centuries',
 94)

Size of data

In [4]:
len(example)

11873

checking only on 3 rows

In [5]:
import pandas as pd
dataframe = pd.DataFrame(example)
df = dataframe.head(3)
df.columns = ['context', 'question', 'is_impossible', 'answer', 'answer_start']
df

,context,question,is_impossible,answer,answer_start
0,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,False,France,159
1,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,False,10th and 11th centuries,94
2,The Normans (Norman: Nourmands; French: Norman...,From which countries did the Norse originate?,False,"Denmark, Iceland and Norway",256


create vocab file, tokenize

go through every file (train and validaton: context and question) and then get distinct words saved

In [6]:
from keras.preprocessing.text import Tokenizer

def tokenize_text(phrase):
    t  = Tokenizer()

    t.fit_on_texts(phrase)

    sequences = t.texts_to_sequences(phrase)
    
    inverse_word_index= {value: key for key, value in t.word_index.items()}

    return list(inverse_word_index.values()), t.word_index, inverse_word_index, sequences


C:\Users\bijno\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
def create_word_vocab(df):
    sentences_list = []

    for eachRow in df.itertuples():
        sentences_list.append(eachRow.context)
        sentences_list.append(eachRow.question)

    return sentences_list

vocab_tokens, word_index, inverse_word_index, sequence = tokenize_text(create_word_vocab(df))
print(vocab_tokens)
print('~~~~~')
print(sequence)
print('~~~~~')
print(len(word_index))
print('~~~~~')
print(inverse_word_index)
print('~~~~~')
print(word_index)


['the', 'and', 'of', 'to', 'in', 'from', 'their', 'normans', 'were', 'norman', 'who', '10th', 'centuries', 'west', 'francia', 'with', 'normandy', 'norse', 'nourmands', 'french', 'normands', 'latin', 'normanni', 'people', '11th', 'gave', 'name', 'a', 'region', 'france', 'they', 'descended', 'comes', 'norseman', 'raiders', 'pirates', 'denmark', 'iceland', 'norway', 'under', 'leader', 'rollo', 'agreed', 'swear', 'fealty', 'king', 'charles', 'iii', 'through', 'generations', 'assimilation', 'mixing', 'native', 'frankish', 'roman', 'gaulish', 'populations', 'descendants', 'would', 'gradually', 'merge', 'carolingian', 'based', 'cultures', 'distinct', 'cultural', 'ethnic', 'identity', 'emerged', 'initially', 'first', 'half', 'century', 'it', 'continued', 'evolve', 'over', 'succeeding', 'what', 'country', 'is', 'located', 'when', 'which', 'countries', 'did', 'originate']
~~~~~
[[1, 8, 10, 19, 20, 21, 22, 23, 9, 1, 24, 11, 5, 1, 12, 2, 25, 13, 26, 7, 27, 4, 17, 28, 29, 5, 30, 31, 9, 32, 6, 18, 1

Define params

In [8]:
# Todo

LENGTH_CONTEXT = 200
LENGTH_QUESTION=10
LENGTH_ANSWER=10


In [9]:
from keras.preprocessing.sequence import pad_sequences
    
def sequence_list(phrase, maxlen, data=df, word_index=word_index):
    seq_list=[]
    token_list = []

    for index, row in data.iterrows():
        t, _, _, _ = tokenize_text([row[phrase]])
        token_list.append(t)
        seq_list.append([word_index[w] for w in t])

    padded_data = pad_sequences(seq_list, maxlen=maxlen)

    return token_list, seq_list, padded_data


In [10]:
tokenizer_answer, seq_list_answer, padded_data_answer = sequence_list(phrase = 'answer', maxlen = LENGTH_ANSWER)

print(tokenizer_answer)
print('~~~~~~~~')

tokenizer_question, seq_list_question, padded_data_question = sequence_list(phrase = 'question', maxlen = LENGTH_QUESTION)

print(seq_list_question)
print('~~~~~~~~')
print(padded_data_question)
print('~~~~~~~~')


tokenizer_context, seq_list_context, padded_data_context = sequence_list(phrase = 'context', maxlen = LENGTH_CONTEXT)

print(seq_list_context)
print('~~~~~~~~')
print(padded_data_context)
print('~~~~~~~~')


[['france'], ['10th', 'and', '11th', 'centuries'], ['denmark', 'iceland', 'and', 'norway']]
~~~~~~~~
[[5, 79, 80, 81, 17, 82], [83, 9, 1, 8, 5, 17], [6, 84, 85, 86, 1, 18, 87]]
~~~~~~~~
[[ 0  0  0  0  5 79 80 81 17 82]
 [ 0  0  0  0 83  9  1  8  5 17]
 [ 0  0  0  6 84 85 86  1 18 87]]
~~~~~~~~
[[1, 2, 3, 4, 5, 7, 6, 8, 10, 9, 11, 12, 13, 14, 15, 16, 19, 20, 21, 22, 23, 24, 25, 26, 27, 17, 28, 29, 30, 31, 32, 18, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78], [1, 2, 3, 4, 5, 7, 6, 8, 10, 9, 11, 12, 13, 14, 15, 16, 19, 20, 21, 22, 23, 24, 25, 26, 27, 17, 28, 29, 30, 31, 32, 18, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78], [1, 2, 3, 4, 5, 7, 6, 8, 10, 9, 11, 12, 13, 14, 15, 16, 19, 20, 21, 22, 23, 24, 25, 26, 27, 17

sequence answer array from the vocab list

In [11]:
import numpy as np

ans_arr=[]
for answers in tokenizer_answer:
    y = np.zeros(len(word_index) + 1)
    for item in answers:
        print(item)
        y[word_index[item]] = 1
    ans_arr.append(y)

print(ans_arr)

pred_arr = np.array(ans_arr)

pred_arr

france
10th
and
11th
centuries
denmark
iceland
and
norway
[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.]), array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.]), array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.

create model

In [12]:
from keras import layers
from keras.layers import Input, Embedding, RepeatVector, Dense
from keras.layers.recurrent import LSTM
from keras.models import Model

hidden_size = 100

sentence_passage = Input(shape=(LENGTH_CONTEXT,), dtype='int32')
encoded_sentence_passage = Embedding(len(word_index)+1, hidden_size)(sentence_passage)

question = Input(shape=(LENGTH_QUESTION,), dtype='int32')
encoded_question = Embedding(len(word_index)+1, hidden_size)(question)
encoded_question = LSTM(hidden_size)(encoded_question)
encoded_question = RepeatVector(LENGTH_CONTEXT)(encoded_question)

merge = layers.add([encoded_sentence_passage, encoded_question])
merge = LSTM(hidden_size,go_backwards=True)(merge)
predicted = Dense(len(inverse_word_index)+1, activation='softmax')(merge)

Instructions for updating:
Colocations handled automatically by placer.


Compile and fit

In [13]:
# ToDo
model = Model([sentence_passage, question], predicted)
model.compile(optimizer='sgd', loss='categorical_crossentropy', )
model.fit( [padded_data_context, padded_data_question], pred_arr, batch_size=1, epochs=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
3/3 [==============================] - 2s 527ms/step - loss: 13.4551


Evaluate

In [14]:
df_test = pd.DataFrame(dataframe.iloc[4]).T
df_test.columns = ['context', 'question', 'is_impossible', 'answer', 'answer_start']
df_test

,context,question,is_impossible,answer,answer_start
4,The Normans (Norman: Nourmands; French: Norman...,What century did the Normans first gain their ...,False,10th,671


In [15]:
vocab_tokens_test, word_index_test, inverse_word_index_test, sequence_test = tokenize_text(create_word_vocab(df_test))

word_index_test

{'the': 1,
 'and': 2,
 'of': 3,
 'their': 4,
 'to': 5,
 'normans': 6,
 'in': 7,
 'from': 8,
 'norman': 9,
 'were': 10,
 'who': 11,
 '10th': 12,
 'centuries': 13,
 'west': 14,
 'francia': 15,
 'with': 16,
 'identity': 17,
 'first': 18,
 'century': 19,
 'nourmands': 20,
 'french': 21,
 'normands': 22,
 'latin': 23,
 'normanni': 24,
 'people': 25,
 '11th': 26,
 'gave': 27,
 'name': 28,
 'normandy': 29,
 'a': 30,
 'region': 31,
 'france': 32,
 'they': 33,
 'descended': 34,
 'norse': 35,
 'comes': 36,
 'norseman': 37,
 'raiders': 38,
 'pirates': 39,
 'denmark': 40,
 'iceland': 41,
 'norway': 42,
 'under': 43,
 'leader': 44,
 'rollo': 45,
 'agreed': 46,
 'swear': 47,
 'fealty': 48,
 'king': 49,
 'charles': 50,
 'iii': 51,
 'through': 52,
 'generations': 53,
 'assimilation': 54,
 'mixing': 55,
 'native': 56,
 'frankish': 57,
 'roman': 58,
 'gaulish': 59,
 'populations': 60,
 'descendants': 61,
 'would': 62,
 'gradually': 63,
 'merge': 64,
 'carolingian': 65,
 'based': 66,
 'cultures': 67,
 'd

In [16]:
_, _, padded_data_question_test = sequence_list(phrase = 'question', maxlen = LENGTH_QUESTION, data = df_test, word_index = word_index_test)

print(seq_list_question)
print('~~~~~~~~')
print(padded_data_question)
print('~~~~~~~~')

_, _, padded_data_context_test = sequence_list(phrase = 'context', maxlen = LENGTH_CONTEXT, data = df_test, word_index = word_index_test)

print(seq_list_context)
print('~~~~~~~~')
print(padded_data_context)
print('~~~~~~~~')


[[5, 79, 80, 81, 17, 82], [83, 9, 1, 8, 5, 17], [6, 84, 85, 86, 1, 18, 87]]
~~~~~~~~
[[ 0  0  0  0  5 79 80 81 17 82]
 [ 0  0  0  0 83  9  1  8  5 17]
 [ 0  0  0  6 84 85 86  1 18 87]]
~~~~~~~~
[[1, 2, 3, 4, 5, 7, 6, 8, 10, 9, 11, 12, 13, 14, 15, 16, 19, 20, 21, 22, 23, 24, 25, 26, 27, 17, 28, 29, 30, 31, 32, 18, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78], [1, 2, 3, 4, 5, 7, 6, 8, 10, 9, 11, 12, 13, 14, 15, 16, 19, 20, 21, 22, 23, 24, 25, 26, 27, 17, 28, 29, 30, 31, 32, 18, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78], [1, 2, 3, 4, 5, 7, 6, 8, 10, 9, 11, 12, 13, 14, 15, 16, 19, 20, 21, 22, 23, 24, 25, 26, 27, 17, 28, 29, 30, 31, 32, 18, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,

Predicts the probability of answer from the word_index

In [17]:
ans=model.predict([padded_data_context_test,padded_data_question_test])

print(ans, len(ans[0]), type(ans))
print('~~~~`')
print("prediction: ", inverse_word_index_test.get(ans.argmax()))

[[0.01133863 0.0109193  0.01144018 0.01095803 0.01138768 0.01113397
  0.01149548 0.01117544 0.01145334 0.01162995 0.01158531 0.01122922
  0.01175249 0.01170729 0.01123376 0.01103042 0.01141837 0.01121611
  0.01120491 0.01150394 0.01107252 0.01166896 0.01137938 0.01129903
  0.01166917 0.01168458 0.01139489 0.0110339  0.01134506 0.01140233
  0.01136044 0.01143155 0.01109564 0.01162125 0.01150065 0.01129172
  0.01130632 0.01159576 0.01146255 0.01160004 0.01134275 0.01139465
  0.01144834 0.01159172 0.0110982  0.01160903 0.01149249 0.01131669
  0.01143707 0.01089211 0.01110673 0.01160846 0.01140953 0.01123598
  0.01147363 0.01152219 0.01129311 0.01103824 0.01142149 0.01106279
  0.01148623 0.01123265 0.01128522 0.01133939 0.01132272 0.01106387
  0.01127423 0.01134555 0.01152323 0.01149854 0.01162649 0.01113039
  0.01153156 0.01110622 0.01127761 0.01156342 0.01167452 0.01148697
  0.01127249 0.01156499 0.01152734 0.01142406 0.01121476 0.01135552
  0.01110801 0.01107397 0.01136229 0.01150295]] 